In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import csv
import numpy as np # linear algebra
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['SAMPLE_SUBMISSION.CSV', 'test_long.csv', 'train_long.csv', 'train.csv', 'test.csv']


**Get a dataframe from file**

In [2]:
def get_dataframe(path_file, training):
    data_file_delimiter = ','
    largest_column_count = 0

    # Loop the data lines
    with open(path_file) as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\n')
        for row in spamreader:
            column_count = len(row[0].split(','))
            largest_column_count = column_count if largest_column_count < column_count else largest_column_count
    
    column_names = []
    if training:
        column_names = ["id_player", "played_race"] + [i for i in range(0, largest_column_count-2)]
    else:
        column_names = ["played_race"] + [i for i in range(0, largest_column_count-1)]
    return pd.read_csv(path_file, header=None, sep=',', names=column_names)

**Get training dataset and preprocessing**

In [24]:
df_training = get_dataframe("../input/train.csv/TRAIN.CSV", training=True)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (3949,3950,3951,3952,3953,3954,3955,3956,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,3989,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034,4035,4036,4037,4038,4039,4040,4041,4042,4043,4044,4045,4046,4047,4048,4049,4050,4051,4052,4053,4054,4055,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096,4097,4098,4099,4100,4101,4102,4103,4104,4105,4106,4107,4108,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118,4119,4120,4121,4122,4123,4124,4125,4126,4127,4128

In [25]:
df_training.describe()

,id_player,played_race,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,10497,10498,10499,10500,10501,10502,10503,10504,10505,10506,10507,10508,10509,10510,10511,10512,10513,10514,10515,10516,10517,10518,10519,10520,10521,10522,10523,10524,10525,10526,10527,10528,10529,10530,10531,10532,10533,10534,10535,10536
count,3052,3052,3044,3044,3044,3044,3044,3044,3043,3042,3042,3042,3042,3042,3042,3041,3041,3041,3040,3040,3040,3040,3039,3039,3039,3039,3039,3039,3039,3039,3039,3039,3038,3037,3037,3037,3037,3037,3037,3037,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
unique,200,3,4,10,17,22,28,27,29,32,30,33,33,32,33,34,35,35,35,36,35,35,34,33,33,34,34,35,35,35,34,34,34,35,34,34,35,34,35,37,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,http://kr.battle.net/sc2/en/profile/2348639/1/...,Protoss,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,s,...,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,hotkey52,hotkey12,t5765,hotkey52,hotkey12,hotkey52,hotkey12,hotkey22,hotkey12,hotkey22,hotkey12,hotkey22,hotkey12,hotkey22,hotkey12,hotkey22,hotkey12,hotkey22,hotkey42,hotkey12,t5770
freq,58,1210,1839,2757,2569,1770,1610,1174,1046,888,868,929,985,958,974,969,1027,1019,1053,1072,1092,1053,1088,1070,1075,1087,1092,1072,1067,1097,1039,1085,1050,1104,1027,1081,1034,1041,1041,1093,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [26]:
df_training.head()

,id_player,played_race,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,10497,10498,10499,10500,10501,10502,10503,10504,10505,10506,10507,10508,10509,10510,10511,10512,10513,10514,10515,10516,10517,10518,10519,10520,10521,10522,10523,10524,10525,10526,10527,10528,10529,10530,10531,10532,10533,10534,10535,10536
0,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,s,s,t5,Base,s,hotkey30,hotkey00,t10,t15,t20,t25,t30,t35,s,hotkey32,s,hotkey32,s,hotkey32,s,t40,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,t45,hotkey32,SingleMineral,s,t50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://eu.battle.net/sc2/en/profile/3074362/1/...,Protoss,s,Base,s,s,Base,s,s,Base,s,t5,hotkey30,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,t10,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,t15,s,hotkey32,s,hotkey32,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,Base,s,hotkey30,hotkey00,t5,t10,t15,s,hotkey32,s,hotkey32,s,hotkey32,s,t20,hotkey32,s,s,hotkey32,s,hotkey32,t25,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://eu.battle.net/sc2/en/profile/3074362/1/...,Protoss,Base,s,s,Base,s,s,s,t5,Base,s,hotkey30,s,hotkey32,s,hotkey32,t10,s,hotkey32,s,hotkey32,t15,s,hotkey32,s,t20,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,t25,s,hotkey32,s,hotkey32,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,Base,s,hotkey30,hotkey00,t5,t10,t15,s,hotkey32,s,hotkey32,s,hotkey32,t20,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,t25,hotkey32,t30,s,hotkey32,t35,s,hotkey32,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Replace categorical values of id_player and played_race by numerical values

In [5]:
##dictionnaries 
#key : ''
id_player_unique_values = pd.Series(df_training.id_player.unique())
played_race_unique_values = pd.Series(df_training.played_race.unique())

#key : 'Protoss', value : unique_id like 0
id_player_from_categ_to_num = {value: index for value, index in zip(id_player_unique_values, range(0,len(id_player_unique_values)))}
played_race_from_categ_to_num = {value: index for value, index in zip(played_race_unique_values, range(0,len(played_race_unique_values)))}

temp = df_training.replace(played_race_from_categ_to_num)
df_training = temp.replace(id_player_from_categ_to_num)

In [27]:
id_player_unique_values

0      http://eu.battle.net/sc2/en/profile/4234852/1/...
1      http://eu.battle.net/sc2/en/profile/3074362/1/...
2      http://eu.battle.net/sc2/en/profile/3401218/1/...
3      http://eu.battle.net/sc2/en/profile/2896854/1/...
4      http://eu.battle.net/sc2/en/profile/3538115/1/...
5      http://eu.battle.net/sc2/en/profile/250458/1/V...
6      http://eu.battle.net/sc2/en/profile/3973341/1/...
7      http://eu.battle.net/sc2/en/profile/2452136/1/...
8      http://eu.battle.net/sc2/en/profile/2222468/1/...
9      http://eu.battle.net/sc2/en/profile/4341883/1/...
10     http://eu.battle.net/sc2/en/profile/950504/1/G...
11     http://eu.battle.net/sc2/en/profile/2898004/1/...
12     http://eu.battle.net/sc2/en/profile/251061/1/L...
13     http://eu.battle.net/sc2/en/profile/1021189/1/...
14     http://eu.battle.net/sc2/en/profile/326029/1/L...
15     http://eu.battle.net/sc2/en/profile/1058669/1/...
16     http://eu.battle.net/sc2/en/profile/1139573/1/...
17     http://eu.battle.net/sc2

In [23]:
print(played_race_unique_values)

0    Protoss
1     Terran
2       Zerg
dtype: object


Replace 't5' 't10' ... values

In [6]:
#replace 't5' 't10' 't15' by NaN
df_training = df_training.replace(to_replace=r'^t\d*', value=np.nan, regex=True)

Compute the frequency of each action for each sample

In [7]:
#define a set of possible actions (like 'Base', 's')
possible_actions = set()
for column in df_training.columns[2:]:
    for action in df_training.loc[:, column].unique():
        if pd.notna(action):
            possible_actions.add(action)

In [8]:
#Associate a numerical value to a categorical value of action
possible_actions_numerical = {value: index for value, index in zip(possible_actions, range(0,len(possible_actions)))}
print(possible_actions_numerical)

{'hotkey71': 0, 'hotkey32': 1, 'hotkey12': 2, 'hotkey91': 3, 'hotkey82': 4, 'hotkey70': 5, 'hotkey60': 6, 'hotkey22': 7, 's': 8, 'hotkey92': 9, 'hotkey51': 10, 'hotkey41': 11, 'hotkey61': 12, 'hotkey81': 13, 'hotkey20': 14, 'hotkey30': 15, 'hotkey50': 16, 'hotkey52': 17, 'hotkey11': 18, 'Base': 19, 'SingleMineral': 20, 'hotkey31': 21, 'hotkey90': 22, 'hotkey62': 23, 'hotkey80': 24, 'hotkey72': 25, 'hotkey21': 26, 'hotkey40': 27, 'hotkey10': 28, 'hotkey42': 29, 'hotkey00': 30, 'hotkey02': 31, 'hotkey01': 32}


In [9]:
#for each sample, we will have the frequency of actions + played_race (+ id_player)
def transform_sample(index, df, training):
    column_index_start = 2 if training else 1
    actions, counts = np.unique(df.iloc[index,column_index_start:].dropna().values, return_counts=True)
    first_part_row = df.iloc[index,0:column_index_start].values
    second_part_row = np.array([count for action, count in zip(actions, counts)])
    row = np.append(first_part_row, second_part_row).reshape(1, -1)
    columns_for_row = ["played_race"] + list(actions)
    if training:
        columns_for_row = ["id_player"] + columns_for_row
    return pd.DataFrame(row, index=[index], columns=columns_for_row)

In [10]:
#compute df of training with only numerical values
df_training_numerical = transform_sample(0, df_training, True)
for index, _ in df_training.iterrows():
    if index != 0:
        df_sample = transform_sample(index, df_training, True)
        df_training_numerical = pd.concat([df_training_numerical, df_sample], sort=False)

In [11]:
#check if all columns are present and replace NaN by 0
columns_to_add = [action for action in possible_actions_numerical.keys() if action not in df_training_numerical.columns]
for column in columns_to_add:
    df_training_numerical.insert(len(df_training_numerical.columns), column, 0)
df_training_numerical = df_training_numerical.fillna(0)

In [12]:
df_training_numerical.head()

,id_player,played_race,Base,SingleMineral,hotkey00,hotkey02,hotkey10,hotkey12,hotkey20,hotkey22,hotkey30,hotkey32,hotkey40,hotkey42,hotkey50,hotkey52,hotkey60,hotkey62,hotkey70,hotkey90,s,hotkey11,hotkey80,hotkey82,hotkey92,hotkey72,hotkey21,hotkey41,hotkey61,hotkey31,hotkey01,hotkey51,hotkey71,hotkey81,hotkey91
0,0,0,66,5,5,41,63,350,37,185,1,401,5,255,4,114,3,18,1,4,674,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,12,0,2,83,65,448,15,215,7,342,3,308,0,0,0,0,0,1,538,3,1,16,18,0,0,0,0,0,0,0,0,0,0
2,0,0,47,3,3,18,17,130,9,126,1,320,7,130,3,88,1,11,0,2,430,0,0,0,2,0,0,0,0,0,0,0,0,0,0
3,1,0,21,0,2,39,32,259,14,70,6,245,3,284,0,0,0,0,0,1,397,1,0,0,14,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,1,0,10,52,6,38,1,200,2,84,1,14,0,0,0,0,208,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Get testing dataset and preprocessing**

In [13]:
df_testing = get_dataframe("../input/test.csv/TEST.CSV", training=False)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6609,6610,6611,6612,6613,6614,6615,6616,6617,6618,6619,6620,6621,6622,6623,6624,6625,6626,6627,6628,6629,6630,6631,6632,6633,6634,6635,6636,6637,6638,6639,6640,6641,6642,6643,6644,6645,6646,6647,6648,6649,6650,6651,6652,6653,6654,6655,6656,6657,6658,6659,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669,6670,6671,6672,6673,6674,6675,6676,6677,6678,6679,6680,6681,6682,6683,6684,6685,6686,6687,6688,6689,6690,6691,6692,6693,6694,6695,6696,6697,6698,6699,6700,6701,6702,6703,6704,6705,6706,6707,6708,6709,6710,6711,6712,6713,6714,6715,6716,6717,6718,6719,6720,6721,6722,6723,6724,6725,6726,6727,6728,6729,6730,6731,6732,6733,6734,6735,6736,6737,6738,6739,6740,6741,6742,6743,6744,6745,6746,6747,6748,6749,6750,6751,6752,6753,6754,6755,6756,6757,6758,6759,6760,6761,6762,6763,6764,6765,6766,6767,6768,6769,6770,6771,6772,6773,6774,6775,6776,6777,6778,6779,6780,6781,6782,6783,6784,6785,6786,6787,6788

In [14]:
df_testing.head()

,played_race,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,6777,6778,6779,6780,6781,6782,6783,6784,6785,6786,6787,6788,6789,6790,6791,6792,6793,6794,6795,6796,6797,6798,6799,6800,6801,6802,6803,6804,6805,6806,6807,6808,6809,6810,6811,6812,6813,6814,6815,6816
0,Zerg,s,s,s,hotkey10,s,hotkey60,s,hotkey00,s,hotkey90,t5,hotkey02,s,hotkey02,hotkey62,hotkey02,s,hotkey02,hotkey62,hotkey02,hotkey62,s,hotkey02,hotkey62,s,hotkey62,hotkey02,hotkey62,hotkey02,hotkey62,s,hotkey02,hotkey62,s,hotkey02,hotkey62,hotkey02,hotkey62,hotkey02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Terran,s,hotkey30,hotkey00,t5,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,s,hotkey32,t10,s,hotkey32,s,hotkey32,s,hotkey32,t15,s,t20,hotkey32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Protoss,Base,s,s,Base,s,hotkey20,hotkey30,hotkey60,hotkey40,hotkey80,hotkey70,s,hotkey62,s,hotkey62,t5,s,hotkey62,s,hotkey20,hotkey62,hotkey22,hotkey62,s,hotkey62,s,hotkey62,hotkey22,hotkey62,t10,s,hotkey62,hotkey22,hotkey62,s,hotkey62,t15,s,hotkey62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zerg,s,s,s,s,hotkey20,s,hotkey40,t5,t10,hotkey22,hotkey42,s,hotkey42,s,t15,s,s,s,hotkey42,s,hotkey42,hotkey22,hotkey42,hotkey22,hotkey42,t20,s,hotkey42,s,hotkey42,s,hotkey42,hotkey22,hotkey42,hotkey22,hotkey42,s,hotkey42,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Protoss,Base,s,s,Base,s,hotkey50,hotkey30,t5,hotkey40,s,t10,hotkey52,t15,s,t20,hotkey52,t25,s,hotkey52,s,hotkey52,s,hotkey52,s,hotkey52,s,hotkey52,s,hotkey52,s,hotkey52,s,hotkey52,s,t30,hotkey52,t35,s,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Preprocessing part

In [15]:
#transform played_race as numerical column and fill with NaN
temp = df_testing.replace(played_race_numerical)
df_testing = temp.replace(to_replace=r'^t\d*', value=np.nan, regex=True)

In [16]:
#compute df of testing with only numerical values
df_testing_numerical = transform_sample(0, df_testing, False)
for index, _ in df_testing.iterrows():
    if index != 0:
        df_sample = transform_sample(index, df_testing, False)
        df_testing_numerical = pd.concat([df_testing_numerical, df_sample], sort=False)

In [17]:
#check if all columns are present and replace NaN by 0
columns_to_add = [action for action in possible_actions_numerical.keys() if action not in df_testing_numerical.columns]
for column in columns_to_add:
    df_testing_numerical.insert(len(df_testing_numerical.columns), column, 0)
df_testing_numerical = df_testing_numerical.fillna(0)

In [18]:
df_testing_numerical.head()

,played_race,Base,SingleMineral,hotkey00,hotkey02,hotkey10,hotkey11,hotkey12,hotkey20,hotkey22,hotkey30,hotkey32,hotkey60,hotkey61,hotkey62,hotkey70,hotkey72,hotkey90,hotkey92,s,hotkey40,hotkey42,hotkey50,hotkey52,hotkey80,hotkey82,hotkey21,hotkey31,hotkey41,hotkey51,hotkey91,hotkey01,hotkey71,hotkey81
0,2,19,3,14,847,7,31,352,10,67,3,2,3,1,981,4,43,6,29,848,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,2,33,14,0,336,31,150,1,467,0,0,0,0,0,1,47,572,3,402,1,157,0,0,0,0,0,0,0,0,0,0
2,0,20,0,1,0,10,0,479,53,628,60,251,1,0,1130,3,32,0,0,556,2,217,0,0,5,49,0,0,0,0,0,0,0,0
3,2,17,2,0,0,68,89,525,13,129,4,30,1,0,3,0,0,0,0,1588,3,528,2,3,0,0,6,2,1,0,0,0,0,0
4,0,21,0,3,0,4,7,74,8,30,11,16,0,0,0,1,3,0,0,236,5,24,1,83,4,1,0,0,0,0,0,0,0,0


**Compute accuracy**

In [19]:
def get_accuracy(labels, predicted_labels):
    compare_result = np.unique(np.equal(labels,predicted_labels), return_counts=True)
    good_predictions = 0
    total_predictions = len(predicted_labels)
    for index, value in enumerate(compare_result[0]):
        if value:
            good_predictions = compare_result[1][index]
    return good_predictions/total_predictions*100   

*Check good transformation from numerical to categorical*

In [ ]:
def get_accuracy_from_categorical(indexes, predicted_labels):
    name_players_random_forest = id_player_unique_values.iloc[indexes]
    
    

**Random forest**

In [20]:
model = RandomForestClassifier(n_estimators=10)
model.fit(df_training_numerical.iloc[:,1:].values, df_training_numerical.id_player.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
#testing
indexes_random_forest = model.predict(df_testing_numerical.values)
name_players_random_forest = id_player_unique_values.iloc[indexes_random_forest]
result_random_forest = pd.DataFrame(name_players_random_forest, columns = ["prediction"])
result_random_forest.index = df_testing_numerical.index
result_random_forest.insert(0, "RowId", result_random_forest.index+1)
result_random_forest.head()

,RowId,prediction
0,1,http://eu.battle.net/sc2/en/profile/4234852/1/...
1,2,http://eu.battle.net/sc2/en/profile/2896854/1/...
2,3,http://eu.battle.net/sc2/en/profile/3074362/1/...
3,4,http://eu.battle.net/sc2/en/profile/1021189/1/...
4,5,http://us.battle.net/sc2/en/profile/3817071/1/...


**Decision Tree**

In [ ]:
#training
#model = tree.DecisionTreeClassifier()
#model = model.fit(df_training_numerical.iloc[:,1:].values, df_training_numerical.id_player.values.reshape(-1, 1))

In [ ]:
#test on training set (should be removed soon)
#indexes_training = model.predict(df_training_numerical.iloc[:,1:].values)
#get_accuracy(df_training_numerical.id_player.values, indexes_training)

In [ ]:
#testing
#indexes = model.predict(df_testing_numerical.values)
#name_players = id_player_unique_values.iloc[indexes]

In [ ]:
#result = pd.DataFrame(name_players, columns = ["prediction"])
#result.index = df_testing_numerical.index
#result.insert(0, "RowId", result.index+1)

**Export a submission file**

In [ ]:
result_random_forest.to_csv("second_result.csv", index=False)